In [152]:
"""
Fix the data frame. At the end, row should have the following columns:
- start: pd.Timestemap
- end: pd.Timestamp
- name: str
- topic: str (python or go)
- earnings: np.float64
"""

import pandas as pd
import numpy as np
import datetime as dt

df = pd.read_csv('workshops.csv')
df

,Year,Month,Start,End,Name,Earnings
0,2021.0,NaN,NaN,NaN,NaN,NaN
1,NaN,June,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,3.0,gRPC in Go,"$33,019"
3,NaN,NaN,7.0,10.0,Optimizing Python,"$42,238"
4,NaN,NaN,28.0,30.0,python Foundations,"$24,372"
5,NaN,July,NaN,NaN,NaN,NaN
6,NaN,NaN,5.0,8.0,go concurrency,"$46,382"
7,NaN,NaN,21.0,22.0,Writing Secure Go,"$27,038"


In [153]:
# cleaning the columns names
df.columns = df.columns.map(lambda x: x.lower())
df

,year,month,start,end,name,earnings
0,2021.0,NaN,NaN,NaN,NaN,NaN
1,NaN,June,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,3.0,gRPC in Go,"$33,019"
3,NaN,NaN,7.0,10.0,Optimizing Python,"$42,238"
4,NaN,NaN,28.0,30.0,python Foundations,"$24,372"
5,NaN,July,NaN,NaN,NaN,NaN
6,NaN,NaN,5.0,8.0,go concurrency,"$46,382"
7,NaN,NaN,21.0,22.0,Writing Secure Go,"$27,038"


In [154]:
# cleaning dates

# detecting rows giving years and months (but nothing else)
date_mask = pd.notna(df['year']) | pd.notna(df['month'])
date_mask
# filling the year and month forward
df[['year', 'month']] = df[['year', 'month']].fillna(method='ffill')
# removing rows that were only indicating a year or month
df = df[~date_mask].reset_index(drop=True)
# converting floats to ints (now that NaNs are removed)
df = df.astype({"year": 'int', 
                "start": 'int',
                'end': 'int'})
df

,year,month,start,end,name,earnings
0,2021,June,1,3,gRPC in Go,"$33,019"
1,2021,June,7,10,Optimizing Python,"$42,238"
2,2021,June,28,30,python Foundations,"$24,372"
3,2021,July,5,8,go concurrency,"$46,382"
4,2021,July,21,22,Writing Secure Go,"$27,038"


In [155]:
# cleaning start and end

def to_date(row, col):
  y = row['year']
  m = row['month']
  d = row[col]
  string = f"{y} {m} {d}"
  return pd.to_datetime(string)

df['start'] = df.apply(to_date, axis=1, args=['start',])
# works as well:
# start = df.apply(lambda row: to_date(row, 'start'), axis=1)
df['end'] = df.apply(to_date, axis=1, args=['end',])
# works as well:
# end = df.apply(lambda row: to_date(row, 'end'), axis=1)
df.drop(columns=['year', 'month'], inplace=True)
df

,start,end,name,earnings
0,2021-06-01,2021-06-03,gRPC in Go,"$33,019"
1,2021-06-07,2021-06-10,Optimizing Python,"$42,238"
2,2021-06-28,2021-06-30,python Foundations,"$24,372"
3,2021-07-05,2021-07-08,go concurrency,"$46,382"
4,2021-07-21,2021-07-22,Writing Secure Go,"$27,038"


In [156]:
# cleaning name

df = df.astype({"name": 'string'})
df.dtypes

start       datetime64[ns]
end         datetime64[ns]
name                string
earnings            object
dtype: object

In [160]:
# adding topics

def get_topic(row):
  if 'python' in row['name'].lower():
    return 'python'
  if 'go' in row['name'].lower():
    return 'go'
  raise ValueError(f'no topic found in name {row["name"]}')

df['topic'] = df.apply(get_topic, axis=1)
df

,start,end,name,earnings,topic
0,2021-06-01,2021-06-03,gRPC in Go,"$33,019",go
1,2021-06-07,2021-06-10,Optimizing Python,"$42,238",python
2,2021-06-28,2021-06-30,python Foundations,"$24,372",python
3,2021-07-05,2021-07-08,go concurrency,"$46,382",go
4,2021-07-21,2021-07-22,Writing Secure Go,"$27,038",go


In [168]:
# cleaning earnings

def clean_earnings(x):
  earnings = str(x)
  earnings = earnings.replace("$", "").replace(",", "")
  return float(earnings)

df['earnings'] = df['earnings'].map(clean_earnings)
df.dtypes

start       datetime64[ns]
end         datetime64[ns]
name                string
earnings           float64
topic               object
dtype: object

In [169]:
# reordering
df = df[['start', 'end', 'name', 'topic', 'earnings']]
df

,start,end,name,topic,earnings
0,2021-06-01,2021-06-03,gRPC in Go,go,33019.0
1,2021-06-07,2021-06-10,Optimizing Python,python,42238.0
2,2021-06-28,2021-06-30,python Foundations,python,24372.0
3,2021-07-05,2021-07-08,go concurrency,go,46382.0
4,2021-07-21,2021-07-22,Writing Secure Go,go,27038.0
